In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.drop(columns=['Unnamed: 32','id'],inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [ ]:
class MySimpleNN():

  def __init__(self,X): # Corrected from __int__
    self.weights = torch.rand(X.shape[1], dtype = torch.float64, requires_grad=True)
    self.bias = torch.zeros(1, dtype = torch.float64, requires_grad=True)

  def forward(self,X):
    z = torch.matmul(X, self.weights) + self.bias # Corrected from torch.matmu
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self, y_pred, y):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)

    loss = -(y * torch.log(y_pred) + (1-y) * torch.log(1 - y_pred)).mean() # Corrected y_train_tensor to y
    return loss

In [ ]:
learning_rate = 0.1
epochs = 25

In [ ]:
model = MySimpleNN(X_train_tensor)

for epochs in range(epochs):

  y_pred = model.forward(X_train_tensor)

  loss = model.loss_function(y_pred, y_train_tensor)

  loss.backward()

  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  model.weights.grad.zero_() # Corrected from model.weight.grad.zero_()
  model.bias.grad.zero_()

  print(f'Epoch: {epochs + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.5039934258780716
Epoch: 2, Loss: 0.4880561767434162
Epoch: 3, Loss: 0.4726115442651977
Epoch: 4, Loss: 0.45764915808120965
Epoch: 5, Loss: 0.44315816326464796
Epoch: 6, Loss: 0.4291273069706077
Epoch: 7, Loss: 0.4155451221125754
Epoch: 8, Loss: 0.40240019316981746
Epoch: 9, Loss: 0.3896814817473861
Epoch: 10, Loss: 0.37737868397958685
Epoch: 11, Loss: 0.36548258878576145
Epoch: 12, Loss: 0.35398540266510153
Epoch: 13, Loss: 0.3428810057707518
Epoch: 14, Loss: 0.33216510048020137
Epoch: 15, Loss: 0.3218352125732126
Epoch: 16, Loss: 0.311890507894016
Epoch: 17, Loss: 0.3023313947080213
Epoch: 18, Loss: 0.29315890637368525
Epoch: 19, Loss: 0.2843738962208569
Epoch: 20, Loss: 0.27597613103615065
Epoch: 21, Loss: 0.2679634216905007
Epoch: 22, Loss: 0.2603309565463587
Epoch: 23, Loss: 0.2530709810238574
Epoch: 24, Loss: 0.24617289055334976
Epoch: 25, Loss: 0.23962370092405655


In [ ]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.9210526347160339
